In [1]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout

# what follows is from the OpenMM script builder
# with some minor modifications...

## Simulation setup

In [2]:
# read in a starting structure for ethane and the
# corresponding force field file
pdb = app.PDBFile('butane.pdb')
forcefield = app.ForceField('butane.gaff2.xml')

# setup system by taking topology from pdb file;
# run gas phase simulation with 2 fs time step (using SHAKE)
# at 323.15 K using a Langevin thermostat (integrator) with
# coupling constant of 5.0 ps^-1
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, 
                                 constraints=app.HBonds)
integrator = mm.LangevinIntegrator(450*unit.kelvin, 5.0/unit.picoseconds, 
                                   2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName('CPU')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

## Energy minimization

This reduces the potential energy of the system before beginning dynamics. This eliminates "bad" (i.e., overly close) contacts and generally leads to more stable simulation behavior.

In [3]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy before minimization is %s" % st.getPotentialEnergy())

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy after minimization is %s" % st.getPotentialEnergy())

Minimizing...
Potential energy before minimization is 5.7975281000609264 kJ/mol
Potential energy after minimization is 5.295941238695407 kJ/mol


## Equilibration

We will run a (very) short equilibration simulation to bring the molecule up to our desired temperature.  If this were a periodic system, we would also aim to bring the density/volume to equilibrium at the desired pressure.

In [4]:
print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(5000)

Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,76.48697370862196,257.8674404566612
200,68.01400503330059,310.98573295230767
300,60.07554302075222,299.9728840043958
400,49.02698368047763,353.3030517580511
500,56.85569491551386,364.244374324878
600,69.98425498249378,283.30463160182677
700,70.02549687477728,419.07328816451854
800,63.365230510123546,344.0750100279498
900,68.37077991457008,468.56015922947563
1000,43.995964155690416,368.43707969157913
1100,47.76317988525799,430.03222362590185
1200,73.07070112409667,346.15546638576456
1300,71.54611549707965,463.09303167107873
1400,57.79740914492643,282.50574253258645
1500,57.91538846018957,411.798846685056
1600,45.19514746685826,255.45948858838491
1700,63.07222550218059,566.5957843203486
1800,69.92606454290157,372.49612685396124
1900,76.88062963583643,459.9985473713956
2000,51.40867606103891,316.77223861389865
2100,66.78986357306847,379.3649831296764
2200,45.22825122285225,380.2872773131634
2300,64.35758857048907,

## Production

Now we run a long MD simulation with parameters that are identical to the equilibration phase (other than simulation length, of course!).  We will also save a trajectory file (i.e., corodinats vs. time) of this simulation that we can analyze afterward using MDTraj (or other trajectory analysis tools).

In [5]:
print('Running Production...')

simulation.reporters.clear()
# output basic simulation information below every 25000 steps/50 ps
simulation.reporters.append(app.StateDataReporter(stdout, 25000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))
# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 50 steps/0.10 ps
simulation.reporters.append(app.DCDReporter('butane_sim.dcd', 50))

# run the simulation for 1.25x10^6 steps/2.5 ns
simulation.step(1250000)
print('Done!')

Running Production...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
25000,50.00000000001514,56.715941235124205,472.01649359812876,0
50000,99.99999999994834,43.34250561027865,469.1104107147512,348
75000,149.99999999998812,63.56153753978836,690.9244489604131,346
100000,200.00000000022686,60.64166836711938,317.1727229788162,347
125000,250.0000000004656,63.860653000967744,429.05673442852503,346
150000,300.00000000070435,71.01029899257655,442.6609860004124,343
175000,350.0000000009431,68.70151424560473,505.3927553455099,340
200000,400.00000000118183,54.252140178429016,248.89510931980823,338
225000,450.0000000014206,53.47342599580898,378.8583601708541,335
250000,500.0000000016593,78.19951242781391,508.6064868225797,334
275000,550.000000000818,57.497256713660704,372.9348357246201,332
300000,599.9999999996356,63.7210387097415,446.9488040538295,331
325000,649.9999999984533,62.4515695369746,529.5549432003434,330
350000,699.999999997271,71.91674289882398,523.